In [18]:

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr
import tempfile
import google.generativeai as genai

# Load environment variables
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"✅ OpenAI API key loaded (starts with {openai_api_key[:8]})")
else:
    print("⚠️ OpenAI API key not found!")

openai = OpenAI()
google.generativeai.configure()
# --- Models ---
MODEL_GPT = "gpt-4o-mini"

# --- System message ---
system_prompt = (
    "You are a knowledgeable teaching assistant who explains code and computer science "
    "concepts clearly, step-by-step, with examples when helpful."
)

# --- Streaming for GPT ---
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True,
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            result += chunk.choices[0].delta.content
            yield result


gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
def gemini_simple(prompt):
    # Prepare system + user messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    # Call Gemini via the OpenAI-compatible API
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=messages
    )

    # Extract and return the text
    return response.choices[0].message.content


def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    # Start streaming completion from Gemini via OpenAI-compatible API
    stream = gemini_via_openai_client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=messages,
        stream=True  # important: enables incremental streaming
    )

    partial_output = ""
    for event in stream:
        # Each chunk has 'delta' content
        if event.choices[0].delta.content:
            partial_output += event.choices[0].delta.content
            yield partial_output

# --- Unified streaming function ---
def stream_answer(question, model_choice):
    prompt = f"Explain the following code snippet or concept:\n{question}"
    if model_choice == "GPT":
        result = stream_gpt(prompt)
    elif model_choice == "Gemini":
        result = stream_gemini(prompt)  
    else:
        raise ValueError("Unknown model")
    yield from result

# --- Text-to-Speech (Audio Output) ---
def text_to_speech(text):
    if not text:
        return None
    speech_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=text
    )
    response.stream_to_file(speech_file.name)
    return speech_file.name

# --- Combined logic for Gradio ---
def full_pipeline( text, model_choice):
   
    answer = ""
    for chunk in stream_answer(text, model_choice):
        answer = chunk

    audio_output = text_to_speech(answer)
    return answer, audio_output

# --- Gradio UI ---
intro = """
#  Code Teaching Assistant  
Ask or speak a Python, coding, or computer science question.  
Choose GPT or Gemini.  
The assistant explains it step-by-step — and speaks back to you!
"""

ui = gr.Interface(
    fn=full_pipeline,
    inputs=[
        gr.Textbox(label=" Type your question/code:", lines=5, placeholder="Paste code or ask a CS question..."),
        gr.Dropdown(["GPT", "Gemini"], label=" Select Model", value="GPT")
    ],
    outputs=[
        gr.Markdown(label=" Explanation"),
        gr.Audio(label=" Spoken Response")
    ],
    title="Code & CS Teaching Assistant",
    description=intro,
    allow_flagging="never",
)

ui.launch(inbrowser=True)


✅ OpenAI API key loaded (starts with sk-proj-)


C:\Users\DELL\anaconda3\envs\llms\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7889
* To create a public link, set `share=True` in `launch()`.


C:\Users\DELL\AppData\Local\Temp\ipykernel_18324\3394524894.py:91: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file.name)
C:\Users\DELL\AppData\Local\Temp\ipykernel_18324\213000604.py:110: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file.name)
